<a href="https://colab.research.google.com/github/IANGECHUKI176/deeplearning/blob/main/pytorch/nlp/pytorch-seq2seq/1_Sequence_to_Sequence_Learning_with_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: [Bentrevett Github](https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb)

paper : [Sequence to Sequence Learning
with Neural Networks](https://arxiv.org/abs/1409.3215)

Preparing Data

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
from tqdm import tqdm
import random
import math
import time

In [ ]:
import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
from torchtext.utils import download_from_url, extract_archive
import io

In [ ]:
# !python -m spacy download en_core_web_sm
# !python -m spacy download de_core_news_sm

In [ ]:

url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
train_urls = ('train.de.gz', 'train.en.gz')
val_urls = ('val.de.gz', 'val.en.gz')
test_urls = ('test_2016_flickr.de.gz', 'test_2016_flickr.en.gz')

train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

def build_vocab(filepath, tokenizer):
    counter = Counter()
    with io.open(filepath, encoding="utf8") as f:
        for string_ in f:
            counter.update(tokenizer(string_))
    v2 = vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])
    v2.set_default_index(0)
    return v2

de_vocab = build_vocab(train_filepaths[0], de_tokenizer)
en_vocab = build_vocab(train_filepaths[1], en_tokenizer)

def data_process(filepaths):
    raw_de_iter = iter(io.open(filepaths[0], encoding="utf8"))
    raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
    data = []
    for (raw_de, raw_en) in zip(raw_de_iter, raw_en_iter):
        de_tensor_ = torch.tensor([de_vocab[token] for token in de_tokenizer(raw_de)],
                                dtype=torch.long)
        en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en)],
                                dtype=torch.long)
        data.append((de_tensor_, en_tensor_))
    return data

train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)

100%|██████████| 637k/637k [00:00<00:00, 10.0MB/s]
100%|██████████| 569k/569k [00:00<00:00, 7.06MB/s]
100%|██████████| 24.7k/24.7k [00:00<00:00, 2.06MB/s]
100%|██████████| 21.6k/21.6k [00:00<00:00, 3.19MB/s]
100%|██████████| 22.9k/22.9k [00:00<00:00, 2.35MB/s]
100%|██████████| 21.1k/21.1k [00:00<00:00, 3.34MB/s]


In [ ]:
BATCH_SIZE = 128
PAD_IDX = de_vocab['<pad>']
BOS_IDX = de_vocab['<bos>']
EOS_IDX = de_vocab['<eos>']

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def generate_batch(data_batch):
  de_batch, en_batch = [], []
  for (de_item, en_item) in data_batch:
    de_batch.append(torch.cat([torch.tensor([BOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
    en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
  de_batch = pad_sequence(de_batch, padding_value=PAD_IDX)
  en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
  return de_batch, en_batch

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)


In [ ]:
class Encoder(nn.Module):
    def __init__(self,input_dim,embed_dim,hidden_dim,n_layers,dropout):
        super(Encoder,self).__init__()
        self.hidden_dim  = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim,embed_dim)
        self.rnn = nn.LSTM(embed_dim,hidden_dim,n_layers,dropout = dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self,src):
        #src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))
        #embedded = [src len, batch size, emb dim]

        out,(hidden,cell) = self.rnn(embedded)
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #outputs are always from the top hidden layer
        return hidden,cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self,output_dim,embed_dim,hidden_dim,n_layers,dropout):
        super(Decoder,self).__init__()
        self.hidden_dim  = hidden_dim
        self.n_layers = n_layers
        self.output_dim = output_dim

        self.embedding = nn.Embedding(output_dim,embed_dim)
        self.rnn = nn.LSTM(embed_dim,hidden_dim,n_layers,dropout = dropout)

        self.fc_out = nn.Linear(hidden_dim,output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,input,hidden,cell):
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        input = input.unsqueeze(0)
        #input = [1,batch size]

        embedded = self.dropout(self.embedding(input))
        #embedded = [1,batch size,embed_dim]
        output , (hidden,cell) = self.rnn(embedded,(hidden,cell))

        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        prediction = self.fc_out(output.squeeze(0))
        #prediction = [batch size, output dim]
        return prediction,hidden,cell

Seq2Seq

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder,device):
        super(Seq2Seq,self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hidden_dim == decoder.hidden_dim ,"Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers ,'No of layer in encoder and decoder must be equal'
    def forward(self,src,trg,teacher_forcing_ratio = 0.5):
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = decoder.output_dim

        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len,batch_size,trg_vocab_size).to(self.device)

        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden,cell = self.encoder(src)

        #first input to the decoder is the <sos> tokens
        input = trg[0,:]

        for t in range(1,trg_len):
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            prediction,hidden,cell = self.decoder(input,hidden,cell)
            outputs[t] = prediction
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            #get the highest predicted token from our predictions
            top1 = prediction.argmax(1)
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
INPUT_DIM = len(de_vocab)
OUTPUT_DIM = len(en_vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
N_LAYERS = 2



encoder = Encoder(INPUT_DIM,ENC_EMB_DIM,ENC_HID_DIM,N_LAYERS,ENC_DROPOUT)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(encoder,decoder,device).to(device)

In [ ]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(19215, 128)
    (rnn): LSTM(128, 256, num_layers=2, dropout=0.6)
    (dropout): Dropout(p=0.6, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(10838, 128)
    (rnn): LSTM(128, 256, num_layers=2, dropout=0.6)
    (fc_out): Linear(in_features=256, out_features=10838, bias=True)
    (dropout): Dropout(p=0.6, inplace=False)
  )
)

In [ ]:
def count_parameters(model:nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,475,350 trainable parameters


In [ ]:
TRG_PAD_IDX = en_vocab.get_stoi()['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, (src, trg) in enumerate(tqdm(iterator)):
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, (src, trg) in enumerate(tqdm(iterator)):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')
train_losses = []
val_losses = []
for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)
    end_time = time.time()

    train_losses.append(train_loss)
    val_losses.append(valid_loss)

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    torch.save(model.state_dict(), 'tut1-model.pt')
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

100%|██████████| 8/8 [00:01<00:00,  6.86it/s]


Epoch: 01 | Time: 0m 57s
	Train Loss: 5.391 | Train PPL: 219.363
	 Val. Loss: 5.019 |  Val. PPL: 151.289


100%|██████████| 8/8 [00:01<00:00,  7.12it/s]


Epoch: 02 | Time: 0m 57s
	Train Loss: 4.861 | Train PPL: 129.181
	 Val. Loss: 4.983 |  Val. PPL: 145.951


100%|██████████| 8/8 [00:01<00:00,  6.84it/s]


...

Epoch: 09 | Time: 0m 57s
	Train Loss: 3.916 | Train PPL:  50.191
	 Val. Loss: 4.632 |  Val. PPL: 102.716


100%|██████████| 8/8 [00:01<00:00,  6.02it/s]


Epoch: 10 | Time: 0m 57s
	Train Loss: 3.842 | Train PPL:  46.617
	 Val. Loss: 4.529 |  Val. PPL:  92.680


100%|██████████| 8/8 [00:01<00:00,  6.63it/s]

| Test Loss: 4.518 | Test PPL:  91.616 |


In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

100%|██████████| 8/8 [00:01<00:00,  6.56it/s]

| Test Loss: 4.520 | Test PPL:  91.795 |


In [ ]:
# Define the translation function
def translate_sentence(model, src_sentence,device, max_len=50):
    model.eval()
    with torch.no_grad():
        if isinstance(src_sentence, str):
            tokens = de_tokenizer(src_sentence)
        else:
            tokens = src_sentence


        tokens = ['<bos>'] + tokens + ['<eos>']
        src_indexes = de_vocab.lookup_indices(tokens)

        src_tensor = torch.tensor(src_indexes).unsqueeze(1).to(device)


        with torch.no_grad():
            hidden,cell = model.encoder(src_tensor)

        BOS_INDEX = en_vocab.get_stoi()["<bos>"]

        trg_indexes = [BOS_INDEX]

        for i in range(max_len):

            trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
            with torch.no_grad():
                prediction,hidden,cell = model.decoder(trg_tensor,hidden,cell)


            pred_token = prediction.argmax(1).item()

            trg_indexes.append(pred_token)

            if pred_token == en_vocab.get_stoi()["<eos>"]:
                break

    trg_tokens = en_vocab.lookup_tokens(trg_indexes)

    return trg_tokens[1:]

In [ ]:
def get_test_data(filepaths): #test_filepaths
    raw_de_iter = iter(io.open(filepaths[0], encoding="utf8"))
    raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
    return raw_de_iter,raw_en_iter

In [ ]:
test_x,test_y = get_test_data(test_filepaths)
src_arr = []
trg_arr = []
for x,y in zip(test_x,test_y):
    src_arr.append(x)
    trg_arr.append(y)

In [ ]:
example_idx = 10
src = src_arr[example_idx]
trg = trg_arr[example_idx]

translation = translate_sentence(model,src, device)
print('src:',src)
print('trg:',trg)
print(f'predicted trg = {translation}')

src: Eine Mutter und ihr kleiner Sohn genießen einen schönen Tag im Freien.

trg: A mother and her young song enjoying a beautiful day outside.

predicted trg = ['A', 'person', 'in', 'a', 'a', 'a', 'a', '.', '\n', '<eos>']


In [ ]:
example_idx = 12
src = src_arr[example_idx]
trg = trg_arr[example_idx]

translation = translate_sentence(model,src, device)
print('src:',src)
print('trg:',trg)
print(f'predicted trg = {translation}')

src: Eine Frau, die in einer Küche eine Schale mit Essen hält.

trg: A woman holding a bowl of food in a kitchen.

predicted trg = ['A', 'woman', 'is', 'a', 'a', 'a', 'a', 'a', '.', '\n', '<eos>']


In [ ]:
example_idx = 14
src = src_arr[example_idx]
trg = trg_arr[example_idx]

translation = translate_sentence(model,src, device)
print('src:',src)
print('trg:',trg)
print(f'predicted trg = {translation}')

src: Drei Leute sitzen in einer Höhle.

trg: Three people sit in a cave.

predicted trg = ['Two', 'people', 'are', 'a', '.', '\n', '<eos>']


In [ ]:
example_idx = 16
src = src_arr[example_idx]
trg = trg_arr[example_idx]

translation = translate_sentence(model,src, device)
print('src:',src)
print('trg:',trg)
print(f'predicted trg = {translation}')

src: Eine Blondine hält mit einem Mann im Sand Händchen.

trg: A blond holding hands with a guy in the sand.

predicted trg = ['A', 'woman', 'in', 'a', 'blue', 'shirt', 'is', 'a', 'a', '.', '.', '\n', '<eos>']
